In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import requests
from bs4 import BeautifulSoup

import time

import pandas as pd

from pytube import YouTube
import re
from datetime import datetime

import sqlite3

In [2]:
def crawling_youtuber_video_link(path, youtuber, num_end=10):
    """해당 유튜버의 영상 링크들을 가져오기
    
    path : 크롬드라이버 주소
    youtuber : 유튜버의 홈 링크
    num_end : 동영상 탭에서 end 키를 누르는 횟수 => 비디오 링크를 얼마나 가져올 것인가
    
    가져오는 변수를 좀 추가해야되나??
    여기서 더 안가져오고 동영상 링크 들어가면 거기서 많이 가져오면 될 것 같기도 하고
    """
    driver = webdriver.Chrome(path)
    driver.maximize_window() # 창 크기 최대화
    
    driver.get(youtuber) # 해당 링크로 이동
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="tabsContent"]/paper-tab[2]/div').click() # 버튼 클릭
    
    # end 키를 눌러 페이지 맨 밑으로 이동
    for i in range(num_end):
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(5)
        
    dom = driver.page_source
    soup = BeautifulSoup(dom, 'html.parser')
    link_source = soup.select("div#details") # 영상들이 있는 부분의 소스 추출
    
    video_link_list = []
    
    for link in link_source:
        title = link.find('a')['title']
        href = 'https://www.youtube.com' + link.find('a')['href']
        video_link_list.append([datetime.now().strftime("%Y/%m/%d %H:%M:%S"), title, href])
    
    driver.close()
    
    youtuber_video_link = pd.DataFrame(video_link_list, columns=['datetime', 'title', 'video_link'])
    
    return youtuber_video_link

In [3]:
def get_video_info(video_link):
    '''pytube를 이용하여 동영상의 제목, 링크, 영상 길이(초), 평점, 썸네일 링크, 조회수, 영상 설명, 자동 생성 자막을 가져오는 함수
    
    video_link : 영상 url
    '''
    
    crawling_time = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
    try:
        # 영상 가져오기
        source = YouTube(video_link)

        # 영상 정보
        title = source.title
        length = source.length
        rating  = source.rating
        thumbnail_url = source.thumbnail_url
        views = source.views
        description = source.description

        # 자동 생성 자막 불러오기 및 전처리
        if 'ko' in source.captions.keys():
            ko_caption = source.captions.get('ko')
            ko_caption_convert_to_srt = ko_caption.generate_srt_captions()

            rule1 = r'\d{1,}\n\d{2}:\d{2}:\d{2}\,\d{3} --> \d{2}:\d{2}:\d{2}\,\d{3}\n'
            rule2 = r'\n'
            rule3 = r'\[\w{1,}\]'
            rule4 = '니다'

            ko_caption_convert_to_srt_ = re.sub(rule4, '니다.',
                                                re.sub(rule3, '',
                                                       re.sub(rule2, '', 
                                                              re.sub(rule1, ' ', ko_caption_convert_to_srt))))
        else:
            ko_caption_convert_to_srt_ = 'No caption in video'

        video_info = [crawling_time, title, video_link, length, rating, thumbnail_url, views, description, ko_caption_convert_to_srt_]
    
    except:
        video_info = [None, None, None, None, None, None, None, None, None]
    
    return video_info

In [4]:
def get_video_infos(video_links):
    '''복수의 video_link에 대해 영상 정보 가져오기
    
    video_links : sequence of video_link
    '''
    video_info_list = []
    
    for video_link in video_links:
        video_info = get_video_info(video_link)
        video_info_list.append(video_info)
        
    video_info_df = pd.DataFrame(video_info_list,
                                 columns=['crawling_time', 'title', 'video_link', 'length', 'rating',
                                          'thumbnail_url', 'views', 'description', 'ko_caption'])
    
    return video_info_df

### 유튜버의 영상 링크들 가져오기

selenium 안써야 일관성 있게 가져올 수 있을거같은데... ㅠㅠ 찾아보다가 잘 모르겠어서 일단 다른거부터 했습니다

In [5]:
ytn_video_list = crawling_youtuber_video_link('chromedriver.exe', 'https://www.youtube.com/user/ytnnews24/featured', num_end=10)

In [6]:
ytn_video_list

,datetime,title,video_link
0,2020/08/03 06:47:59,[날씨] 중부 집중호우 계속...일부 시간당 100mm 폭우 / YTN,https://www.youtube.com/watch?v=EYXIEAgNRyI
1,2020/08/03 06:47:59,충북 중북부에 피해 몰려...300㎜ 더 내린다 / YTN,https://www.youtube.com/watch?v=wocK8FtJ_WE
2,2020/08/03 06:47:59,동부간선도로 성수분기점↔수락지하차도 양방향 통제...출근길 불편 예상 / YTN,https://www.youtube.com/watch?v=3jmdUQtzDXo
3,2020/08/03 06:47:59,시간당 최고 50mm 장대비...'하천 범람 우려' 주민 대피령 / YTN,https://www.youtube.com/watch?v=jil9Y8DHOGw
4,2020/08/03 06:47:59,"백악관 ""미 대선은 11월3일""...바이든, 부통령 후보 내주 발표 / YTN",https://www.youtube.com/watch?v=Psq6aJvHENk
...,...,...,...
293,2020/08/03 06:47:59,"北 매체 ""장마로 300~500mm 많은 비...대책 철저히"" / YTN",https://www.youtube.com/watch?v=eH85o1J1MtM
294,2020/08/03 06:47:59,'행락객 대피 수위 넘었다'...이 시각 임진강 필승교 상황 / YTN,https://www.youtube.com/watch?v=tDlaSspP-KE
295,2020/08/03 06:47:59,[날씨] 서울 호우주의보...수도권·영서 모레까지 250mm↑ / YTN,https://www.youtube.com/watch?v=ormBteZC6YQ
296,2020/08/03 06:47:59,"[자막뉴스] ""번개 칠 때 절대 충전하지 마세요""...전기차 운전자가 꼭 봐야 할 ...",https://www.youtube.com/watch?v=GoYeSDmk-1U


### 해당 링크의 영상 정보 (자막 등)

In [7]:
ytn_video_infos = get_video_infos(ytn_video_list['video_link'][0:100])

In [8]:
ytn_video_infos

,crawling_time,title,video_link,length,rating,thumbnail_url,views,description,ko_caption
0,2020/08/03 06:48:03,[날씨] 중부 집중호우 계속...일부 시간당 100mm 폭우 / YTN,https://www.youtube.com/watch?v=EYXIEAgNRyI,131.0,4.454546,https://i.ytimg.com/vi/EYXIEAgNRyI/maxresdefau...,203.0,[앵커]\n오늘도 중부 지방 곳곳에 돌풍과 벼락을 동반한 집중호우가 쏟아지고 있습니...,오늘도 중부지방 곳곳에 돌풍과 벼락을 동반한 집중호우가 쏟아지고 있습니다. 어떻게...
1,2020/08/03 06:48:06,충북 중북부에 피해 몰려...300㎜ 더 내린다 / YTN,https://www.youtube.com/watch?v=wocK8FtJ_WE,147.0,5.000000,https://i.ytimg.com/vi/wocK8FtJ_WE/maxresdefau...,71.0,[앵커]\n어제 300㎜ 넘는 폭우가 쏟아진 충북 북부 지역의 피해가 이만저만이 아...,에어 대 300m 가 넘는 폭우가 쏟아진 충북 북부 지역의 피해가 이만저만이 아닙...
2,2020/08/03 06:48:08,동부간선도로 성수분기점↔수락지하차도 양방향 통제...출근길 불편 예상 / YTN,https://www.youtube.com/watch?v=3jmdUQtzDXo,176.0,5.000000,https://i.ytimg.com/vi/3jmdUQtzDXo/maxresdefau...,86.0,[앵커]\n중부지역 집중호우로 어제부터 팔당댐 방류가 계속되면서 한강 물 높이도 높...,어제부터 팔당 된 방류가 계속되면서 한강물 높이도 높은 수준을 유지하고 있습니다....
3,2020/08/03 06:48:11,시간당 최고 50mm 장대비...'하천 범람 우려' 주민 대피령 / YTN,https://www.youtube.com/watch?v=jil9Y8DHOGw,154.0,4.466667,https://i.ytimg.com/vi/jil9Y8DHOGw/maxresdefau...,117.0,[앵커]\n현재 경기 남부에는 돌풍과 천둥·번개를 동반한 시간당 50m 안팎의 장대...,현재 경기 남부의 는 돌풍과 천둥 관계를 동반한 시간당 50mm 안팎의 장대비가 ...
4,2020/08/03 06:48:14,"백악관 ""미 대선은 11월3일""...바이든, 부통령 후보 내주 발표 / YTN",https://www.youtube.com/watch?v=Psq6aJvHENk,158.0,4.466667,https://i.ytimg.com/vi/Psq6aJvHENk/maxresdefau...,185.0,[앵커]\n도널드 트럼프 미국 대통령이 대선 연기론을 지폈다가 거센 역풍에 물러선 ...,취업후 미국 대통령이 대선 연계로 늘지 펴다가 거센 역풍에 물러선 가운데 백악관이...
...,...,...,...,...,...,...,...,...,...
95,2020/08/03 06:52:24,유통업계 '적과의 동침'...묘한 전략적 제휴 확산 / YTN,https://www.youtube.com/watch?v=X3Mrx7mRQ8A,122.0,5.000000,https://i.ytimg.com/vi/X3Mrx7mRQ8A/maxresdefau...,3248.0,"[앵커]\n'적과의 동침!' 한때 인기 있었던 영화제목인데요, 요즘 유통업계에서 이...",적과의 동침 한때 인기 있었던 영화 제목 인데요 요즘 유통업계에서 이러한 현상이 ...
96,2020/08/03 06:52:27,민주 '부동산 조속 처리'...통합 '막을 방법 없어' / YTN,https://www.youtube.com/watch?v=w7GijdSXdsM,1261.0,4.533333,https://i.ytimg.com/vi/w7GijdSXdsM/maxresdefau...,4641.0,"■ 진행 : 김영수 앵커, 강려원 앵커\n■ 출연 : 최민희 / 前 더불어민주당 의...",임대차 3 법 등 부동산 관련 법안 처리를 둘러싼 여야의 공방이 계속 가열되고 있...
97,2020/08/03 06:52:30,"""10년 새 종부세 내는 5주택 이상 300% 이상 급증""...종부세법 통과 임박 ...",https://www.youtube.com/watch?v=AY9xdcfeG3I,133.0,4.846154,https://i.ytimg.com/vi/AY9xdcfeG3I/maxresdefau...,4563.0,[앵커]\n부동산 시장 과열을 부추기는 투기 목적의 다주택자에 대한 종합부동산세를 ...,부동산 시장 과열을 부추기는 투기 목적 의 다주택자에 대한 종합부동산세 를 늘리는...
98,2020/08/03 06:52:32,급류에 휩쓸리고 토사에 깔리고...충북서 4명 사망·7명 실종 / YTN,https://www.youtube.com/watch?v=7qLvtKbaJBI,53.0,4.090909,https://i.ytimg.com/vi/7qLvtKbaJBI/maxresdefau...,2302.0,집중호우가 내린 충북에서 4명이 숨지고 7명이 실종됐습니다.\n\n오늘 오전 11시...,4 집중호우가 내린 충북 에서는 4명이 숨지고 7명이 실종된 것으로 조사됐습니다....


In [9]:
print(ytn_video_infos.loc[0, 'video_link'])
print(ytn_video_infos.loc[0, 'ko_caption'])

https://www.youtube.com/watch?v=EYXIEAgNRyI
 오늘도 중부지방 곳곳에 돌풍과 벼락을 동반한 집중호우가 쏟아지고 있습니다. 어떻게 오늘 나까지 시간당 50에서 80mm 의 국지적으로 강한 비가 계속 될 것으로 보여서 추가 비 피해가 우려됩니다. 자세한 상황 기상캐스터 연결해서 알아보겠습니다. 10mm 캐스터 4 서울 상암동에 나와있습니다. 지금 비와 가장 많이 오는 곳이 어딥니까 4 서울은 보시는 것처럼 빗줄기가 약해졌다 도 강해지기를 계속해서 반복하고 있는데요 현재의 가장 강한 비가 내리는 지역은 경계 지역으로 특히 경기도 광주 곤지암 우베 는 현재 시간당 70mm 가 넘는 그야말로 물폭탄이 쏟아지고 있습니다. 이정도면 우선의 써도 소용이 없는 수준이고 요 치열에 제대로 분간하기가 어렵기 때문에 안전사고 위험이 굉장히 높아 질 수 있습니다. 주근깨에 교통안전에 각별히 유의 하시는 게 좋겠습니다. 현재 레이더 화면 상으로 보면 은 이 사선으로 길게 발달한 비구름이 이 스톡 원과 강원지역에 걸쳐져 있습니다. 특히 붉은색의 강한 비 부름이 머무는 곳은 경기 남부와 강원 중부 지역 인데요 시간당 30에서 50mm 에 강한 비가 이 지역에 집중되고 있습니다. 이에 따라서 수도권과 강원도 천국 북부지역의 후 경보가 발효 중이고요 중부서 애는 바람이 강하게 불면서 밤사이 강 풍 주 의 보 도 내려진 상태입니다. 일단 내일까지 의 중서부 많은 곳에 300mm 이상 영동과 경북 북부지역 으로도 100mm 가 넘는 많은 비가 더 내리겠습니다. 특히 오네 나까지 중부지방을 중심으로 돌풍과 벼락을 동반한 시간당 100mm 이상의 물폭탄이 쏟아지는 곳이 있겠습니다. 설상 가상으로 태풍이 물건 수 전기 장마 전선의 조금은 힘을 실어 주고 있는데요 수요일까지 중부지방에 500mm 이상의 누적 강우량을 기록한 은 곳도 있겠습니다. 계속해서 주의하시는 게 좋겠습니다. 오늘 낮까지 중부지방에 많고 강한 비가 계속해서 쏟아질 것으로 보입니다. 입이 집안이 많이

In [10]:
print(ytn_video_infos.loc[50, 'video_link'])
print(ytn_video_infos.loc[50, 'ko_caption'])

https://www.youtube.com/watch?v=cdglPt9ND5U
 요즘 아프리카 케냐 젊은이들 사이에서는 마스크도 패션이라는 인식이 퍼지고 있습니다. 결혼식 6 마스크가 등장하는가 하면 자신에게 어울리는 마스크를 주문 제작해서 저마다의 개성을 살리는 사람들이 들고 있는데요 그런 알고 가 만들어낸 신종 유행 송 치즈 리포터가 카메라에 담았습니다. 맞춤 양복을 찾으러 온 케냐 모델 토니 씨 몸에 꼭 맞춘 양복을 입고 재봉사 건넨 마스크까지 착용 해 봅니다. 양복 원단 그리고 보니 와 똑같은 마스크까지 쓴 이 비로소 요즘 케냐에서 유행하는 이름나 빨 맞춤 패션이 완성 됐습니다. 마치 cml 2 po 4 liq tooling 인데 아울 실제 날 쳐 1 단계 7 i 떠안는 다른 모양 라이라 하게 집 란마 익히게 최근 케냐 젊은이들 사이에서는 자신의 개성을 살린 마스크를 착용하는 게 유행처럼 번지고 있습니다. 마스크의 자신이 좋아하는 축구팀 로고를 새겨 넣기도 하고 예비 신부는 결혼식 때 착용할 마스크를 주문제작 가기도 합니다. 화장을 하는 여성들을 위한 마스크도 윤기 입니다. i use 캅스 점에 카펫 벌레 구미 스프 기해서 안락 거 잖아 연소와 있겠는가 물건 대 내외장 있게 get 안습 수 안쪽 800원 콘트롤러 야 문제를 키울수 마스크를 쓰지 않으면 우리 돈 약 23만 원의 벌금을 물리는 캐년 하지만 복원 용 마스크 공급 부족과 수입 마스크 가격 폭등으로 마스크를 직접 만들어 팔고 사는 사람들이 부쩍 많아졌습니다. 마침 거리에서 마스크 상인을 만나 저도 직접 구매해봤습니다. 펜 아이스 연하 다만 1억 타임 뭐 엄마 니노미야 아 메뉴 메뉴 마스크 그 우리 돈 약 1100원 으로 구매 안 마스크입니다. 색상과 디자인은 정말 화려한데 - 를 막아주는 필터가 있는지 비말 차단 효과가 어느 정도인지는 전혀 알 수가 없습니다. 문화를 되게 에 좀 중요시하기 생각하고 잡음 시민이 받으세요 좀 그런게 좀 이런 패션 마스크 쪽으로 가면 된거 아닌가 케냐 업체에서 만드

In [11]:
print(ytn_video_infos.loc[90, 'video_link'])
print(ytn_video_infos.loc[90, 'ko_caption'])

https://www.youtube.com/watch?v=bqGtsnn5EGk
 어제부터 오늘까지 중부지방에는 300mm 가까운 물폭탄이 쏟아졌습니다. 장마와 북상하는 태풍 의 영향으로 내일까지 300mm 이상의 큰 비가 더 내릴 것으로 보여 주위가 필요합니다. 자세한 호와 태풍 상황 알아봅니다. 홈 사실 캐스터 4 기상센터 입니다. 내일 오전까지 중부지방에 강한 비가 이어준다 고요 네 그렇습니다. 오늘 낮 동안에도 경기와 영서 충청 지역을 중심으로 시간당 30에서 40mm 안팎의 강한 비가 이어졌는데요 지금도 강원도 양구에 는 시간당 30mm 가 넘는 장대비가 내리고 있고요 밤사이에 도 계속해서 중부지방에는 국지성 호가 쏟아지게 씁니다. 따라서 서울 등 수도권과 영서 충북 북부 경북 북부 n 호 경보가 발효 중이고요 그밖에 중부 지방으로 는 호우주의보가 이어지고 있습니다. 밤사 에 비해 강도가 더 강해질 것으로 보입니다. 중부 일부 지역에는 시간당 100mm 이상의 물폭탄이 쏟아지는 곳도 있겠는데요 취약 시간대에 강한 비가 집중되는 만큼 계속해서 주의해주시기 바랍니다. 한편 4호 태풍 하구 빛의 경로가 다소 변경 되었습니다. 화요일 쯤 중국 남동쪽 해안에 상륙한 데 세력이 약화 할 것으로 전망 되는데요 다만 중국 내륙에서 10대 저압으로 약화 하면서 다량의 수증기가 우리나라 유입돼 가능성이 있습니다. 우선 내일까지 중서부 많은 곳에 300mm 이상의 큰 비가 내리겠고 경북 북부의 도 100mm 이상의 큰 비가 더 내리겠습니다. 이번 비는 남북의 폭이 좁아 지역간의 강소 타가 크고 한 곳에 집중되는 특징에 있습니다. 비가 잠시 약해 지더라도 다시 국제적으로 강한 비가 쏟아질 수 있는 만큼 계속해서 김장 해 주시기 바랍니다. 지금까지 기장 센터 였습니다.


### db 생성해서 table로 넣기

In [12]:
conn = sqlite3.connect('./youtube_data.db')
cur = conn.cursor()

In [13]:
ytn_video_list.to_sql('ytn_video_list', conn)

In [14]:
ytn_video_infos.to_sql('ytn_video_info', conn)

In [15]:
cur.close()
conn.close()

어떻게 하기는 했는데...ㅠㅠㅠㅠ 뭔가 너무 못한거같아서 ㅠㅠㅠㅠ 완성도가 떨어져서... 정말 죄송합니다 ㅠㅠ

일단 보면서 느낀건 영상 자동 생성 자막에 부정확한 부분이 많아서 그대로 사용하면 되는건가? 라는 생각이 들었고

그래서 주요 뉴스 사이트 들어가보니까 정보 제공 현황이 아래와 같았습니다.

ytn : 영상 설명에 내용 O, 원문 보기 링크 O

kbs : 영상 설명에 내용 O, 원문 보기 링크 X

mbc : 영상 설명에 내용 X, 원문 보기 링크 O

sbs : 영상 설명에 내용 X, 원문 보기 링크 O

연합뉴스 : 영상 설명에 내용 O, 원문 보기 링크 X

그래서 해당 채널별로 따로 코드를 짜서 크롤링을 해야되나?? 라는 생각이 들었고

만약에 케이스를 나눠서 하면 원문 링크가 있으면 거기 들어가서 가져오는 식으로 짜면 될 것 같은데

원문 링크가 없고 영상 설명에만 있으면 그걸 가져온다 했을 때 영상 길이가 길면 영상 설명에 있는 대사를 중략에서 올리더라고요

그래서 영상 설명에 있는 내용을 가져오는 방식으로 하려면 영상 길이에 제약을 둬야 할 것 같네요 (ex. 5분 이하로?)

그리고 네이버 뉴스, 다음 뉴스 요약본 몇 개 보면서 느낀 건

기사 전체를 뜯어서 핵심 내용 파악해서 문장을 재구성해서 요약하는게 아니고

기사 전체의 첫 번째 문장 + 아래의 내용에서 중요하다고 판단되는 문장 2~3개를 그대로 가져오는 방식이더라고요.

저희가 의도하는 것도 그런 방식?으로 진행되는건지 이야기해보면 좋을 것 같다고 생각했어요.